In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

I'm making this project with TensorFlow 2.0 so I need to specify that for Google Colab since it should be compatible with Colab.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Importing required libraries

In [ ]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/flags/flag.data'

dataset_path = tf.keras.utils.get_file("train.csv", data_url)

column_names = ['Name','Landmass','Zone','Area','Population', 'Language', 
                'Religion', 'Bars', 'Stripes', 'Colours', 'Red', 'Green', 
                'Blue', 'Gold', 'White', 'Black', 'Orange', 'Main Hue',
                'Circles', 'Crosses', 'Saltires', 'Quarters', 'Sunstars',
                'Crescent', 'Triangle', 'Icon', 'Animate', 'Text', 'Top Left',
                'Bottom Right']

raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "Unknown", sep=",")

dataset = raw_dataset.copy()
dataset.tail()


Importing dataset from UCI's database.